# Gradient Boosting (XGBoost)

**Rossmann Store Sales**
Forecast sales using store, promotion, and competitor data

This is a Kaggle competition and can be found [here](https://www.kaggle.com/competitions/rossmann-store-sales/overview)

In this notebook I use `Gradient Boosting` implemented in `XGBoost` to predict `Sales`.

The validation RMSE is `????`
On the full train set, Root Mean Square Percentage Error is `????`

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# from scipy.stats import randint as sp_randint 

# my functions
from utils import RMSPE, print_best_model_metrics
from pipeline import full_pipeline

# Load data

In [2]:
# Load the dataset
def load_ds(path: Path, filename: str) -> pd.DataFrame:
    """Read the dataset csv file as a pandas dataframe."""
    return pd.read_csv(path / filename)

# Load dataset
dataset_path = Path().absolute() / "data"
filename = "X_y_train.csv"
X_y_train = load_ds(dataset_path, filename)

X_train = X_y_train # include "Sales", CombinedAttributesAdder() drops it
y_train = X_y_train.loc[:, "Sales"].copy()

print(f"Shape X_y_train: {X_y_train.shape}")
print(f"Shape X_train:   {X_train.shape}")
print(f"Shape y_train:   {y_train.shape}")

Shape X_y_train: (675470, 18)
Shape X_train:   (675470, 18)
Shape y_train:   (675470,)


# Gradient Boosting